In [4]:
import pandas as pd
from openai import OpenAI, AzureOpenAI
import re
import os
import time
from typing import Dict, Any

os.environ['OPENAI_API_VERSION'] = '2024-02-15-preview'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://upappliance.openai.azure.com'
os.environ['AZURE_OPENAI_API_KEY'] = '4gWgDnxVjVaIRQhPJFn2seBhFGhsgfZhpedg6J0lmWDdsltOAlD8JQQJ99AKACYeBjFXJ3w3AAABACOGylYc'
os.environ['AZURE_OPENAI_API_DEPLOYMENT_NAME'] = "up_gpt4o"

# 프롬프트 정의 함수
def create_evaluation_prompt(generated_summary: str, components: str, keyword: str) -> str:
    """키워드 적합성 평가 프롬프트 생성"""
    
    prompt = f"""

[역할 부여]
당신은 고객 피드백 데이터를 분석하고 분류하는 엄격한 전문가입니다. 
당신의 주요 임무는 주어진 영문 요약문과 제품군 정보에 대해 한글 키워드가 얼마나 정확하게 할당되었는지를 평가하는 것입니다.
주어진 입력에 근거해 판단하고, 아래의 편향 억제 규칙을 따르세요.

[편향 억제 규칙]
1. 자신(모델)의 능력, 정체성, 훈련 데이터에 대한 언급은 금지합니다. 개인적인 의견이나 감정을 배제하고, 오직 데이터에 기반한 분석을 수행하세요.
2. 텍스트의 길이나 복잡성에 가점을 주지 마세요. 모든 입력은 동등하게 취급되어야 합니다.
3. 주어진 정보 외에 추가적인 가정을 하지 마세요.

[작업 목표]
제공되는 'generated_summary'(영문 요약), 'components'(제품군), 'keyword'(한글)를 종합적으로 분석하여, keyword가 해당 제품의 요약문 핵심 주제를 정확하게 나타내는지 판단해 주세요.

[평가 기준]
다음 4가지 차원에서 키워드의 적합성을 평가하세요:

1. **관련성(Relevance, 1-5점)**: 키워드가 요약문의 핵심 주제와 얼마나 관련이 있는가?
   - 5점: 요약문의 핵심 주제를 정확히 반영
   - 4점: 요약문의 주요 내용과 강한 관련성
   - 3점: 요약문과 적당한 관련성
   - 2점: 요약문과 약한 관련성
   - 1점: 요약문과 관련성이 거의 없음

2. **특이성(Specificity, 1-5점)**: 키워드가 얼마나 구체적이고 명확한가?
   - 5점: 매우 구체적이고 명확한 키워드
   - 4점: 구체적이고 이해하기 쉬운 키워드
   - 3점: 적당히 구체적인 키워드
   - 2점: 다소 모호하거나 일반적인 키워드
   - 1점: 매우 모호하고 일반적인 키워드

3. **제품 적합성(Product Fit, 1-5점)**: 키워드가 해당 제품군(components)에 얼마나 적합한가?
   - 5점: 해당 제품에 완벽히 적합한 기능/특성
   - 4점: 해당 제품에 적합한 기능/특성
   - 3점: 해당 제품에 가능한 기능/특성
   - 2점: 해당 제품에 다소 부적합한 기능/특성
   - 1점: 해당 제품과 전혀 맞지 않는 기능/특성

4. **포괄성(Coverage, 1-5점)**: 키워드가 요약문의 전체 의미를 얼마나 포괄하는가?
   - 5점: 요약문의 전체 의미를 완전히 포괄
   - 4점: 요약문의 주요 의미를 잘 포괄
   - 3점: 요약문의 일부 의미를 적절히 포괄
   - 2점: 요약문의 일부 의미만 부분적으로 포괄
   - 1점: 요약문의 의미를 거의 포괄하지 못함

[평가 단계]
1. 'generated_summary'의 내용을 완벽하게 파악합니다. 고객이 무엇을 원하고 어떤 문제나 제안을 하는지 핵심을 파악하세요.
2. 'components' 정보를 통해 어떤 제품에 대한 피드백인지 이해합니다.
3. 'keyword'의 의미를 정확하게 이해하고, 위 4가지 평가 기준에 따라 점수를 매깁니다.
4. 종합적인 적합성을 판단하고, 평균 점수를 계산합니다 (평균 점수 3.0 이상이면 적합, 미만이면 부적합).
5. 부적합한 경우 더 나은 키워드를 제안합니다.
6. 더 나은 키워드를 제시할 경우, 그 이유를 'generated_summary'와 'keywords'를 기반으로 설명합니다.
7. 반드시 Output 형식에 맞춰 응답합니다.

[Input 정보]
- Generated Summary: {generated_summary}
- Components: {components}
- Keyword: {keyword}

[Output 형식]
아래 형식으로 정확히 응답해주세요:

**평가 분석:**
요약문 핵심: [요약문의 핵심 내용 요약]
제품 맥락: [제품군에서 고려해야 할 사항]
키워드 분석: [현재 키워드의 의미와 특징]

**점수 평가:**
- 관련성: [점수]/5 - [근거]
- 특이성: [점수]/5 - [근거]  
- 제품 적합성: [점수]/5 - [근거]
- 포괄성: [점수]/5 - [근거]
- 평균 점수: [평균]/5

**최종 판단:**
적합성: [적합/부적합]
이유: [종합적인 판단 근거]
제안 키워드: [부적합한 경우에만 더 나은 키워드 제안, 적합하면 "없음"]
제안 키워드 이유 : [부적한 경우의 제안 키워드를 제안할 때 그 이유를 'generated_summary'와 'keyword'를 기반으로 설명, 만약 적합한 경우여서, 제안 키워드가 없다면 "없음"]
"""
    return prompt


# 응답에서 정보 추출 함수
def parse_evaluation_result(evaluation_text: str) -> Dict[str, Any]:
    """GPT 응답에서 구조화된 정보 추출"""
    
    result = {}
        # 점수 추출
    relevance_match = re.search(r"관련성:\s*(\d+)/5", evaluation_text)
    specificity_match = re.search(r"특이성:\s*(\d+)/5", evaluation_text)
    product_fit_match = re.search(r"제품 적합성:\s*(\d+)/5", evaluation_text)
    coverage_match = re.search(r"포괄성:\s*(\d+)/5", evaluation_text)
    average_match = re.search(r"평균 점수:\s*(\d+\.?\d*)/5", evaluation_text)
        
    result["relevance_score"] = int(relevance_match.group(1)) if relevance_match else None
    result["specificity_score"] = int(specificity_match.group(1)) if specificity_match else None
    result["product_fit_score"] = int(product_fit_match.group(1)) if product_fit_match else None
    result["coverage_score"] = int(coverage_match.group(1)) if coverage_match else None
    result["average_score"] = float(average_match.group(1)) if average_match else None
        
    # 적합성 판단
    suitable_match = re.search(r"적합성:\s*(적합|부적합)", evaluation_text)
    result["is_suitable"] = suitable_match.group(1) == "적합" if suitable_match else None

     # 최종 판단 이유 추출
    reason_match = re.search(r"이유:\s*(.+?)(?=제안 키워드:|$)", evaluation_text, re.DOTALL)
    result["judgment_reason"] = reason_match.group(1).strip() if reason_match else None
        
        
    # 제안 키워드 추출
    suggested_match = re.search(r"제안 키워드:\s*(.+?)(?:\n|$)", evaluation_text)
    suggested_keyword = suggested_match.group(1).strip() if suggested_match else None
    result["suggested_keyword"] = suggested_keyword if suggested_keyword != "없음" else None
        
    return result

def evaluate_keywords_and_save(csv_path: str, sample_size: int = 10, output_file: str = 'evaluation_results_2.csv'):
    """키워드 평가하고 결과를 DataFrame으로 저장"""
    
    # OpenAI 클라이언트 초기화
    client = AzureOpenAI(
				    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
				    api_key=os.environ['AZURE_OPENAI_API_KEY'],
				    api_version=os.environ['OPENAI_API_VERSION']
				)

    
    # 데이터 로드
    df = pd.read_csv(csv_path, encoding='utf-8')
    
    # 필요한 컬럼만 선택
    required_columns = ['generated_summary', 'components', 'keyword']
    df_filtered = df[required_columns]
    
    # # 샘플링
    # if sample_size and sample_size < len(df_filtered):
    #     df_sample = df_filtered.sample(n=sample_size, random_state=42)
    # else:
    #     df_sample = df_filtered
    
    df_sample = df_filtered.iloc[:3000]
    
    print(f"📊 {len(df_sample)}개 키워드 평가 시작...")
    
    results = []
    
    for i, (idx, row) in enumerate(df_sample.iterrows(), 1):
        print(f"🔄 {i}/{len(df_sample)} 평가 중: {row['keyword']}")
        
        # 프롬프트 생성
        prompt = create_evaluation_prompt(
            row['generated_summary'], 
            row['components'], 
            row['keyword']
        )
        
        try:
            # GPT API 호출
            response = client.chat.completions.create(
									    model=os.environ['AZURE_OPENAI_API_DEPLOYMENT_NAME'],  # 배포명 사용 (예: "gpt4o")
									    messages=[{"role": "user", "content": prompt}],
									    temperature=0.1,
									    max_tokens=1000
										)
            
            # 응답 텍스트 추출
            gpt_response = response.choices[0].message.content.strip()
            
            # 응답 파싱
            parsed_result = parse_evaluation_result(gpt_response)
            
            # 결과 저장
            result = {
                'generated_summary': row['generated_summary'],
                'components': row['components'], 
                'keyword': row['keyword'],
                'gpt_response': gpt_response,
                **parsed_result
            }
            
            results.append(result)
            
        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            # 오류가 발생해도 기본 정보는 저장
            results.append({
                'generated_summary': row['generated_summary'],
                'components': row['components'],
                'keyword': row['keyword'],
                'gpt_response': f"오류: {str(e)}",
                'relevance_score': None,
                'specificity_score': None,
                'product_fit_score': None,
                'coverage_score': None,
                'average_score': None,
                'is_suitable': None,
                'suggested_keyword': None
            })
    
    # DataFrame 생성 및 저장
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    print(f"✅ 평가 완료! 결과가 '{output_file}'에 저장되었습니다.")
    print(f"📊 총 {len(results_df)}개 결과 저장")
    
    return results_df


csv_path = '/datasets/DTS2025000182/data/all_origin_updated.csv' # CSV 파일 경로 설정

results_df = evaluate_keywords_and_save(csv_path) # 샘플 크기 조정 가능 None으로 설정 시 전체 데이터 사용

📊 3000개 키워드 평가 시작...
🔄 1/3000 평가 중: 버튼설정
🔄 2/3000 평가 중: 애플홈킷연동
🔄 3/3000 평가 중: 3rd_Party_연동
🔄 4/3000 평가 중: 원격기능유지
🔄 5/3000 평가 중: 타임싱크
🔄 6/3000 평가 중: 타임싱크
🔄 7/3000 평가 중: 타임싱크
🔄 8/3000 평가 중: 타임싱크
🔄 9/3000 평가 중: 내부조명시간설정/변경
🔄 10/3000 평가 중: LCD배경테마바꾸기
🔄 11/3000 평가 중: 침구말림개선
🔄 12/3000 평가 중: 홈공유
🔄 13/3000 평가 중: 필터교체알림
🔄 14/3000 평가 중: 필터청소시기알림
🔄 15/3000 평가 중: 필터청소시기알림
🔄 16/3000 평가 중: 내부조명시간설정/변경
🔄 17/3000 평가 중: 주름개선코스
🔄 18/3000 평가 중: 나만의코스
🔄 19/3000 평가 중: 앱에서종료음재생
🔄 20/3000 평가 중: 펫케어코스
🔄 21/3000 평가 중: 음원변경
🔄 22/3000 평가 중: 음원변경
🔄 23/3000 평가 중: 계절별테마
🔄 24/3000 평가 중: 나만의코스
🔄 25/3000 평가 중: KeepFresh
🔄 26/3000 평가 중: 앱내기능설명
🔄 27/3000 평가 중: 나만의코스
🔄 28/3000 평가 중: 나만의코스
🔄 29/3000 평가 중: 코스명관리
🔄 30/3000 평가 중: 앱개선(편집기능)
🔄 31/3000 평가 중: 나만의코스
🔄 32/3000 평가 중: 나만의코스
🔄 33/3000 평가 중: 나만의코스
🔄 34/3000 평가 중: 나만의코스
🔄 35/3000 평가 중: 건조온도설정/변경
🔄 36/3000 평가 중: 나만의코스
🔄 37/3000 평가 중: 원격기능유지
🔄 38/3000 평가 중: 나만의코스
🔄 39/3000 평가 중: 나만의코스
🔄 40/3000 평가 중: 나만의코스
🔄 41/3000 평가 중: 나만의코스
🔄 42/3000 평가 중: 커스텀멜로디설정/변경
🔄 43/3000 평가 중:

In [1]:
import pandas as pd

# ===== 절대경로 =====
PATH_EVAL = "./evaluation_results_1.csv"        # 기준(원본 컬럼 유지)
PATH_ALL  = '/datasets/DTS2025000182/data/all_origin_updated.csv'          # 200k 풀데이터
OUT_PATH  = "./evaluation_results_1_with_id.csv"

# 1) 로드 (ALL은 필요한 컬럼만)
eval_df = pd.read_csv(PATH_EVAL, low_memory=False)
all_df  = pd.read_csv(PATH_ALL, usecols=["generated_summary", "ticket_id_hashed"], low_memory=False)

# 2) 키 전처리 (NaN 유지 + 공백 제거)
eval_df["generated_summary"] = eval_df["generated_summary"].astype("string").str.strip()
all_df["generated_summary"]  = all_df["generated_summary"].astype("string").str.strip()

# 3) 키 테이블 정리 (NULL 키 제거 + 중복 방지)
key_df = all_df.dropna(subset=["generated_summary"]).drop_duplicates(subset=["generated_summary"], keep="first")

# 4) 기준 파일과 left-merge (원래 컬럼 유지 + ticket_id_hashed 추가)
merged = eval_df.merge(key_df, on="generated_summary", how="left")

# 5) 새 컬럼을 맨 앞으로 이동
cols = ["ticket_id_hashed"] + [c for c in merged.columns if c != "ticket_id_hashed"]
merged = merged[cols]

# 6) 저장
merged.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
print(f"Saved: {OUT_PATH} | rows={len(merged)} | no_match={merged['ticket_id_hashed'].isna().sum()}")

Saved: ./evaluation_results_1_with_id.csv | rows=3000 | no_match=0
